# Medicago GWAS Pipeline

Given a set of trait files (cleaned and normlised), perform a standard GWAS + population structure analysis. For more background, see Medicago_FirstGWAS_Test.

### Trait Preparation  

We assume that the trait files are saved in a folder named "Traits". See examples.

Analyses will be performed in PLINK thus, we assume that there is a trait (phenotype) file that conforms to plink's specifications. Specifically: {FID, IID, trait_one}. Each column should be TAB or SPACE separated. For more info:  https://www.cog-genomics.org/plink2/input#pheno

Plink allows you to create a master trait file containing all the traits that you want to test. However, we haven't tested this yet and the pipeline has not been designed to handle this yet. We recommend that you create a separate trait file for each trait.

### Genotypes

We assume that the Genotypes are either VCF or BCF files, saved in a folder named "Genotypes". Genotypes may be split into separate files per chromosome, or one large file.

The default QC parameters are as follows:  

  - reject SNPs with call frequency < 90%  
  - reject SNPS with minor allele frequency (MAF) < 0.03  
  
These parameters can be changed in the Setup section below. This pipeline will handle the rest for you.

### GWAS  

This pipeline will:  

  - plot the trait data (saved in Results/Visualisations)  
  - determine the population structure  
  - perform a standard GWAS adjusted for population structure, using plink  
  - the results will be saved in Results/  
  - a manhattan plot for each trait will be saved in Results/Visualisations  
  
### PLINK Setup  

You will need PLINK to run this pipeline. To get PLINK up and running perform the following:  

  1. Download the appropriate version of plink from here: https://www.cog-genomics.org/plink2  
  2. Unpack the zipped plink folder somewhere convenient  
  3. Make sure you set the plink_directory paramter in the Setup section below  
  

### Running the pipeline  

Review the Setup section below:  

  1. Change the working directory as appropriate (to the directory where you have the PLINK, Genotypes and Traits folders)  
  2. Review the Genotype QC parameters, and change as appropriate (defaults are usually sufficient)  
  3. Run each cell as required  
  3. Have fun :)  
  
---------------------------------

## Setup  

Change the parameters below as required.

In [1]:
# set the home & plink directory (please no spaces or funny characters)
home_ <- "/Data/Analyses/Pipeline"
plink_ <- "/Data/Analyses/Pipeline/PLINK/plink"

# Genotype QC settings
parameters <- list(
    call_rate = 0.10,                       # rejects SNPs with more than this number of missing calls
    maf = 0.03,                             # rejects SNPs with a MAF < this threshold
    pop_struct_dimensions = 5               # the number of principal components for population structure
)

## Housekeeping  

Run the following cell, it will install required pacakges, test for plink and setup the working directory.

In [2]:
# install & load packages
options(repos = "http://cran.uk.r-project.org")
if (!require(data.table)) {    
    install.packages("data.table")
}
if (!require(gridExtra)) {
    install.packages("gridExtra")
}
if (!require(ggplot2)) {
    install.packages("ggplot2")
}
if (!require(qqman)) {
    install.packages("qqman")
}

library(ggplot2)
library(data.table)
library(gridExtra)
library(qqman)


# setup the working directory
setwd(home_)

if (!dir.exists("./Results")) {
    dir.create(file.path(home_, "Results"))
}
if (!dir.exists("./Results/Visualisations")) {
    dir.create(file.path(home_, "Results/Visualisations"))
}
if (!dir.exists("./ScratchSpace")) {
    dir.create(file.path(home_, "ScratchSpace"))
}


# test plink installation
if (!capture.output({ cat(system(sprintf("%s --version", plink_))) }) == 0) {
    
    cat(sprintf("
        --------------------------------------------------------------------------------

                                      !!!! WARNING !!!! 
        
        PLINK not found. Please review the plink_ directory in the Setup section.

        --------------------------------------------------------------------------------
    ", plink_))
    
}

# extract trait files and genotype files  
trait_files <- list.files("./Traits")
genotype_files <- list.files("./Genotypes")

if (any(c(length(trait_files) < 1, length(genotype_files) < 1))) {
    
    cat(sprintf("
        --------------------------------------------------------------------------------

                                      !!!! WARNING !!!! 
        
        Something has gone wrong with the trait and/or the genotype files. Please review.

        There are %s trait files and %s genotype files. 

        --------------------------------------------------------------------------------
    ", length(trait_files), length(genotype_files)))
    
} else {
    "Ready to proceed..."
}

Loading required package: data.table
Loading required package: gridExtra
Loading required package: ggplot2
Loading required package: qqman

For example usage please run: vignette('qqman')

Citation appreciated but not required:
Turner, S.D. qqman: an R package for visualizing GWAS results using Q-Q and manhattan plots. biorXiv DOI: 10.1101/005165 (2014).



[1] "Ready to proceed..."

## Trait Visualisation  

If you would like to visualise the traits, please run the following cell. Visualisations will be saved in Results/Visualisations.

NOTE: this is not required to run the GWAS, so you may skip it if you like.

In [53]:
for (file_ in trait_files) {
    
    tmp <- fread(sprintf("%s/Traits/%s", home_, file_))
    
    # skipping the FID and IID columns, plot the distribution of traits  
    for (trait_ in colnames(tmp)[-c(1,2)]) {
        
        print(trait_)
        print(str(tmp))
        g1 <- qplot(x= trait_, y = tmp[[trait_]], data = tmp,
                    geom = "boxplot", color = "steelblue") + ggtitle(trait_) + theme_minimal()
        g2 <- qplot(tmp[[trait_]], geom = "histogram", 
                    color = "steelblue", fill = "steelblue", alpha = 0.5) + theme_minimal()              
        g3 <- qplot(tmp[[trait_]], geom = "density", 
                    color = "steelblue", fill = "steelblue", alpha = 0.5) + theme_minimal()
        
        g <- arrangeGrob(g1, g2, g3, ncol = 2, layout_matrix = cbind(c(1, 1), c(2, 3)))        
        ggsave(file = sprintf("%s/Results/Visualisations/%s_%s.png",
                              home_, 
                              strsplit(file_, "\\.")[[1]][1],
                              trait_),
               plot = g,
               width = 800, height = 600, units = c("mm"))
    }
}

[1] "FTa1"
Classes ‘data.table’ and 'data.frame':	106 obs. of  3 variables:
 $ FID : int  0 0 0 0 0 0 0 0 0 0 ...
 $ IID : chr  "HM001" "HM002" "HM003" "HM004" ...
 $ FTa1: num  0.0131 0.0001 0.0001 0.0321 0.0101 0.00883 0.00111 0.0434 0.0001 0.0043 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.


[1] "FTa2"
Classes ‘data.table’ and 'data.frame':	90 obs. of  3 variables:
 $ FID : int  0 0 0 0 0 0 0 0 0 0 ...
 $ IID : chr  "HM003" "HM008" "HM009" "HM011" ...
 $ FTa2: num  -5.83 -6.91 -5.25 -6.91 -3.5 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.


[1] "FIRSTFLWR"
Classes ‘data.table’ and 'data.frame':	67 obs. of  3 variables:
 $ FID      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ IID      : chr  "HM002" "HM010" "HM012" "HM016" ...
 $ FIRSTFLWR: int  103 108 108 115 78 98 71 111 120 111 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.


[1] "StantonFloweringDate"
Classes ‘data.table’ and 'data.frame':	250 obs. of  3 variables:
 $ FID                 : int  0 0 0 0 0 0 0 0 0 0 ...
 $ IID                 : chr  "HM001" "HM002" "HM003" "HM004" ...
 $ StantonFloweringDate: num  -6.29 -3.29 2.08 1.58 2.28 2.33 2.39 2.45 -4.67 0.08 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.


[1] "VRN_FT1"
Classes ‘data.table’ and 'data.frame':	110 obs. of  3 variables:
 $ FID    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ IID    : chr  "HM002" "HM003" "HM005" "HM006" ...
 $ VRN_FT1: num  -1.34 -0.66 -2.32 -4.45 -3.15 ...
 - attr(*, ".internal.selfref")=<externalptr> 
NULL


`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.


## GWAS Pipeline  

Because we have a different number of accessions in each trait file, we will have to run this pipeline separately for each trait. For each trait then, we will run the following:  

  - for each chromosome:  
    - perform QC  
    - estimate population structure  
    - perform association study  
    
A plot of the population structure for each chromosome will be saved in Results/Visualisations. The results of the association study will be saved in Results. A final manhattan plot will be saved in Results/Visualisations, as well as visualised here.  

In [ ]:
setwd(sprintf("%s/ScratchSpace", home_))

for (trait_ in trait_files) {
    
    cat(sprintf("

    #
    # Trait: %s
    #

    ", trait_))
    
    pheno_file <- sprintf("%s/Traits/%s", home_, trait_)
    lcl_trait <- strsplit(trait_, "\\.")[[1]][1]
    
    for (geno_ in genotype_files) {
        
        chromosome <- strsplit(geno_, "-")[[1]][1]
        assoc_results <- sprintf("%s/Results/%s_%s", home_, chromosome, lcl_trait)
        
        # QC steps
        cat(sprintf("
            Performing QC steps on genotype file: %s
        ", geno_))
        
        geno_file <- sprintf("%s/Genotypes/%s", home_, geno_)
        cmd <- sprintf("

        %s --bcf %s \\
              --const-fid \\
              --allow-extra-chr \\
              --keep %s \\
              --geno %s \\
              --maf %s \\
              --make-bed \\
              --out qc_genotypes

        ", plink_, geno_file, pheno_file, parameters$call_rate, parameters$maf
        )
        
        write(cmd, file = "plink.cmd")
        system("bash plink.cmd")
        
        # population structure
        cat("Estimating population structure...")
        cmd <- sprintf("%s --bfile %s --pca 5 --out population_structure", plink_, "qc_genotypes")
        system(cmd)
    
        # Association test
        cat("Performing association test...")
        cmd <- sprintf("

        %s --bfile %s \\
                    --assoc \\
                    --pheno %s \\
                    --covar %s \\
                    --covar-name V3, V4, V5, V6, V7 \\
                    --allow-no-sex \\
                    --out %s
        ", plink_, "qc_genotypes", pheno_file, "population_structure.eigenvec", assoc_results)
        system(cmd)
        
    }
}